<img src='https://upload.wikimedia.org/wikipedia/commons/thumb/a/aa/Logo_DuocUC.svg/2560px-Logo_DuocUC.svg.png' width=50%, height=20%>

# Introducción 
 

# Regresión Logística - Validación
$$y = \beta_0 + x_1\beta_1 + ... + x_k\beta_k$$

$$p = f(y)= \frac{1}{1+e^{-y}} = \frac{1}{1+e^{-(\beta_0 + x_1\beta_1 + ... + x_k\beta_k)}}$$
 


 # Actividades 
 ESTABILIDAD DE UNA MATRIZ ELÉCTRICA

La estabilidad en una matriz energética del país es de suma importancia para el progreso económico y el bienestar de sus habitantes (solo tener presente la sensación de inseguridad, malestar, etc. que se puede generar con un corte no previsto puede afectar el mercado económico y el bienestar de una población)

El desafio es tratar de determinar si la red es estable (columna ```stabf```) basado en un conjunto de caracteristicas disponibles:


1. tau[x]: reaction time of participant (real from the range [0.5,10]s). Tau1 - the value for electricity producer. 
2. p[x]: nominal power consumed(negative)/produced(positive)(real). For consumers from the range [-0.5,-2]s^-2; p1 = abs(p2 + p3 + p4) 
3. g[x]: coefficient (gamma) proportional to price elasticity (real from the range [0.05,1]s^-1). g1 - the value for electricity producer. 
4. stab: the maximal real part of the characteristic equation root (if positive - the system is linearly unstable)(real) 
5. stabf: the stability label of the system (categorical: stable/unstable) 


Estos datos fueron usados en el paper: "Towards Concise Models of Grid Stability"

https://archive.ics.uci.edu/ml/datasets/Electrical+Grid+Stability+Simulated+Data+

¡Empezamos! 



## Cargamos el dataset

In [ ]:
import pandas as pd
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/00471/Data_for_UCI_named.csv')

df.head()

In [ ]:
df.isna().sum()

In [ ]:
df.stabf.unique()

In [ ]:
df.stabf.value_counts()/df.shape[0]

## Preparamos el dataset

¿Por qué hay variables que se borran?

Explica la celda siguiente...

In [ ]:
X = df.drop(['stab','stabf'],axis=1)
Y = df.stabf.copy()

## Cargamos y Ajustamos el modelo

In [ ]:
import numpy as np
from sklearn.linear_model import LogisticRegression

mo1 = LogisticRegression(solver='lbfgs')

mo1.fit(X, Y)

In [ ]:
Yhat = mo1.predict(X)
Yprob = mo1.predict_proba(X)

In [ ]:
Yhat

In [ ]:
Yprob

## Evaluamos los resultados

Explica que significan cada una de las métricas siguientes, en cuanto al valor que están mostrando.

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(Y, Yhat))

Explica que está indicando la matriz de confusión siguiente

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(Y, Yhat)

In [ ]:
from sklearn.preprocessing import LabelEncoder
enc = LabelEncoder()
Y1 = enc.fit_transform(Y)
Yhat1 = enc.fit_transform(Yhat)


In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(Y1,Yhat1)

In [ ]:
print(Yhat[0:10])
print(Yhat1[0:10])
print(Yprob[0:10,])

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(Y1,Yprob[:,1])

## Será necesario escalar los datos?

Esto se hace ya que las características son completamente distintas en magnitudes, unidades y rango por lo que lo mejor es escalarlos para llevarlos a un mismo nivel de magnitudes.

Para realizar este procedimiento importaremos el método StandarScaler.

INVESTIGA!....¿por qué es necesario ESCALAR? ¿cómo funciona el método StandardScaler? ¿y el MinMaxScaler? ¿Hay otros? ¿Cuáles?

####Escalamiento (opción 1)

In [3]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)
Xscaled = scaler.transform(X)


NameError: ignored

¿Hubo algún cambio en la verificación de las métricas?...revisa las siguientes celdas....

In [ ]:
mo1 = LogisticRegression(solver='lbfgs')
mo1.fit(Xscaled, Y)
Yhat = mo1.predict(Xscaled)

print(classification_report(Y, Yhat))

print(confusion_matrix(Y, Yhat))

####Escalamiento (opción 2)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scalerN = MinMaxScaler()
scalerN.fit(X)
XscaledN = scalerN.transform(X)

mo1 = LogisticRegression(solver='lbfgs')
mo1.fit(XscaledN, Y)
Yhat = mo1.predict(XscaledN)

print(classification_report(Y, Yhat))

print(confusion_matrix(Y, Yhat))

¿Hubo algún cambio en las métricas? ¿ qué indican estas?

## Evitando el Overfitting

In [ ]:
from sklearn.model_selection import train_test_split
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X,Y,test_size=0.2)

## Re-entrenamos y evaluamos los resultados

In [ ]:
import numpy as np
from sklearn.linear_model import LogisticRegression

mo1 = LogisticRegression(solver='lbfgs')

mo1.fit(Xtrain, Ytrain)

Yhat = mo1.predict(Xtest)
Yprob = mo1.predict_proba(Xtest)

from sklearn.metrics import classification_report
print(classification_report(Ytest, Yhat))

Explica cada una de las métricas anteriores. ¿Qué ocurre con el modelo?

## Sintonización del Modelo

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()

params = {"max_iter": [50,100,150,200],
          "solver": ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
         }
grid = GridSearchCV(estimator=model, param_grid=params,cv=5,scoring='accuracy')
grid.fit(Xtrain, Ytrain,)

In [ ]:
print(grid.best_score_)
print(grid.best_params_)

In [ ]:
pd.DataFrame(grid.cv_results_).iloc[grid.best_index_]

In [ ]:
Yhat = grid.predict(Xtest)
Yprob = grid.predict_proba(Xtest)

from sklearn.metrics import classification_report
print(classification_report(Ytest, Yhat))

## CONCLUSIONES

¿Es este el mejor modelo? ¿Por qué?

Reflexiona sobre el ejercicio y realiza una explicación que puedas compartir con el curso.